# Amazon Personalize HRNN-Metadata + Contextual Recommendations Example


## Introduction <a class="anchor" id="intro"></a>

For the most part, the algorithms in Amazon Personalize (called recipes) look to solve different tasks, explained here:

1. **HRNN & HRNN-Metadata** - Recommends items based on previous user interactions with items.
1. **HRNN-Coldstart** - Recommends new items for which interaction data is not yet available.
1. **Personalized-Ranking** - Takes a collection of items and then orders them in probable order of interest using an HRNN-like approach.
1. **SIMS (Similar Items)** - Given one item, recommends other items also interacted with by users.

No matter the use case, the algorithms all share a base of learning on user-item-interaction data which is defined by 3 core attributes:

1. **UserID** - The user who interacted
1. **ItemID** - The item the user interacted with
1. **Timestamp** - The time at which the interaction occurred


## Choose a dataset or data source <a class="anchor" id="source"></a>
[Back to top](#top)

As we mentioned, the user-item-iteraction data is key for getting started with the service. This means we need to look for use cases that generate that kind of data, a few common examples are:

1. Video-on-demand applications
1. E-commerce platforms
1. Social media aggregators / platforms

There are a few guidelines for scoping a problem suitable for Personalize. We recommend the values below as a starting point, although the [official limits](https://docs.aws.amazon.com/personalize/latest/dg/limits.html) lie a little lower.

* Authenticated users
* At least 50 unique users
* At least 100 unique items
* At least 2 dozen interactions for each user 

Most of the time this is easily attainable, and if you are low in one category, you can often make up for it by having a larger number in another category.

Generally speaking your data will not arrive in a perfect form for Personalize, and will take some modification to be structured correctly. This notebook looks to guide you through all of that. 

To begin with, we are going to use an airlines review dataset. A scraped dataset created from all user reviews found on Skytrax (www.airlinequality.com). The data can be found at https://github.com/quankiquanki/skytrax-reviews-dataset 

In [274]:
import pandas as pd, numpy as np
import io
import scipy.sparse as ss
import json
import time
import datetime
import os
import sagemaker.amazon.common as smac
import boto3
import uuid
from botocore.exceptions import ClientError

### Import and Explore your dataset

In [275]:
data_dir = "airlines_data"
!mkdir $data_dir
!cd $data_dir && wget https://raw.githubusercontent.com/quankiquanki/skytrax-reviews-dataset/master/data/airline.csv


--2020-06-18 16:33:44--  https://raw.githubusercontent.com/quankiquanki/skytrax-reviews-dataset/master/data/airline.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.64.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.64.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34752262 (33M) [text/plain]
Saving to: ‘airline.csv’

airline.csv         100%[===================>]  33.14M   113MB/s    in 0.3s    

2020-06-18 16:33:45 (113 MB/s) - ‘airline.csv’ saved [34752262/34752262]



In [276]:
airline_df = pd.read_csv(data_dir + '/airline.csv')
airline_df.head()

,airline_name,link,title,author,author_country,date,content,aircraft,type_traveller,cabin_flown,route,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,wifi_connectivity_rating,value_money_rating,recommended
0,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,D Ito,Germany,2015-04-10,Outbound flight FRA/PRN A319. 2 hours 10 min f...,NaN,NaN,Economy,NaN,7.0,4.0,4.0,4.0,0.0,NaN,NaN,4.0,1
1,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Ron Kuhlmann,United States,2015-01-05,Two short hops ZRH-LJU and LJU-VIE. Very fast ...,NaN,NaN,Business Class,NaN,10.0,4.0,5.0,4.0,1.0,NaN,NaN,5.0,1
2,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,E Albin,Switzerland,2014-09-14,Flew Zurich-Ljubljana on JP365 newish CRJ900. ...,NaN,NaN,Economy,NaN,9.0,5.0,5.0,4.0,0.0,NaN,NaN,5.0,1
3,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Tercon Bojan,Singapore,2014-09-06,Adria serves this 100 min flight from Ljubljan...,NaN,NaN,Business Class,NaN,8.0,4.0,4.0,3.0,1.0,NaN,NaN,4.0,1
4,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,L James,Poland,2014-06-16,WAW-SKJ Economy. No free snacks or drinks on t...,NaN,NaN,Economy,NaN,4.0,4.0,2.0,1.0,2.0,NaN,NaN,2.0,0


As we can see here the dataset has a lot of columns we can use to create the required data sets in Amazon Personalize.

The first thing we are going to do is make 2 copies of the dataset

In [277]:
a_interactions_df = airline_df.copy()
a_users_df = airline_df.copy()

## Building the Interactions Data set

Let's build the interactions dataset. By following the these steps:

- Drop the columns we are not interested in
- Create a new column to account for Event Type
- Rename the columns to a more standard naming convention for you Amazon Personalize import job



In [278]:
# Keeping only 5 columns
a_interactions_df = a_interactions_df[['airline_name', 'author', 'date', 'cabin_flown', 'overall_rating']]
# Creating an additional column for Event Type
a_interactions_df['EVENT_TYPE']='RATING'
# Making sure the author name is unique without spaces
a_interactions_df['author'] = a_interactions_df['author'].str.replace(" ","")
# Rename the columns to a more Amazon Personalize standar notation
a_interactions_df.rename(columns = {'airline_name':'ITEM_ID', 'author':'USER_ID',
                              'date':'TIMESTAMP', 'cabin_flown': 'CABIN_TYPE', 'overall_rating': 'EVENT_VALUE'}, inplace = True) 
a_interactions_df.head()

,ITEM_ID,USER_ID,TIMESTAMP,CABIN_TYPE,EVENT_VALUE,EVENT_TYPE
0,adria-airways,DIto,2015-04-10,Economy,7.0,RATING
1,adria-airways,RonKuhlmann,2015-01-05,Business Class,10.0,RATING
2,adria-airways,EAlbin,2014-09-14,Economy,9.0,RATING
3,adria-airways,TerconBojan,2014-09-06,Business Class,8.0,RATING
4,adria-airways,LJames,2014-06-16,Economy,4.0,RATING


Amazon Personalize supports **contextual recommendations**, through which you can improve relevance of recommendations by generating them within a context, for instance device type, location, time of day, etc. Contextual information is also useful in personalization for new/unidentified users even when the past interactions of these users are not known.

In our case we are going to use **Cabin Type** as a context to recommend which airline is the best fit for our user. Let's explore which values we are going to be able to pass as our context when getting recommendations


In [279]:
a_interactions_df.CABIN_TYPE.unique()

array(['Economy', 'Business Class', nan, 'Premium Economy', 'First Class'],
      dtype=object)

As we can see our current **Timestamp** value in the dataset is a string. Amazon Personalize requires the timestamp value as Unix type. Let's take a random timestamp value and convert it to Unix type

In [280]:
# Get a random value from the timestamp column
arb_time_stamp = a_interactions_df.iloc[50]['TIMESTAMP']
# Transform this string to date time
date_time_obj = datetime.datetime.strptime(arb_time_stamp, '%Y-%m-%d')
print('Date:', date_time_obj.date())
# Get the date of this object
d = date_time_obj.date()
# Transform the date object to Unix time
unixtime = time.mktime(d.timetuple())
print('Unix Time: ', unixtime)

Date: 2004-03-18
Unix Time:  1079568000.0


Now we are going to do the same transformation to all of our values in the timestamp column

In [281]:
# Define a function
def convert_to_unix(string_date):
    date_time_obj = datetime.datetime.strptime(string_date, '%Y-%m-%d')
    d = date_time_obj.date()
    return time.mktime(d.timetuple())

# Apply this function across the Timestamp column
a_interactions_df['TIMESTAMP'] = a_interactions_df['TIMESTAMP'].apply(convert_to_unix)
a_interactions_df.head(5)

,ITEM_ID,USER_ID,TIMESTAMP,CABIN_TYPE,EVENT_VALUE,EVENT_TYPE
0,adria-airways,DIto,1.428624e+09,Economy,7.0,RATING
1,adria-airways,RonKuhlmann,1.420416e+09,Business Class,10.0,RATING
2,adria-airways,EAlbin,1.410653e+09,Economy,9.0,RATING
3,adria-airways,TerconBojan,1.409962e+09,Business Class,8.0,RATING
4,adria-airways,LJames,1.402877e+09,Economy,4.0,RATING


Let's take a look at some of our dataset properties

In [282]:
a_interactions_df.describe()

,TIMESTAMP,EVENT_VALUE
count,4.139600e+04,36861.000000
mean,1.373950e+09,6.039527
std,5.771909e+07,3.214680
min,0.000000e+00,1.000000
25%,1.350864e+09,3.000000
50%,1.389658e+09,7.000000
75%,1.412122e+09,9.000000
max,1.438474e+09,10.000000


Are there any Null values??

In [283]:
a_interactions_df.isnull().values.any()

True

Let's drop those Null values and make sure there aren't any

In [284]:
a_interactions_df = a_interactions_df.dropna()
a_interactions_df.isnull().values.any()

False

Now that we have our data ready for Amazon Personalize, let's save it into a file locally

In [285]:
interactions_filename = "a_interactions.csv"
a_interactions_df.to_csv((data_dir + "/"+interactions_filename), index=False, float_format='%.0f')

## Building the Users Data set

Let's build the users dataset. By following the these steps:

- Drop the columns we are not interested in
- Create a new column to account for Nationality as user metadata
- Rename the columns to a more standard naming convention for you Amazon Personalize import job


In [286]:
# Copy the complete airlines data set
a_users_df = airline_df.copy()
# Select only interested columns
a_users_df = a_users_df[['author', 'author_country']]
# Clean up the authors string
a_users_df['author'] = a_users_df['author'].str.replace(" ","")
# Rename the columns
a_users_df.rename(columns = { 'author':'USER_ID', 'author_country':'NATIONALITY'}, inplace = True) 
# Drop any null values
a_users_df = a_users_df.dropna()
# Save your file locally
users_filename = "a_users.csv"
a_users_df.to_csv((data_dir +"/"+users_filename), index=False)

## Configure an S3 bucket and an IAM  role <a class="anchor" id="bucket_role"></a>
[Back to top](#top)

So far, we have downloaded, manipulated, and saved the data onto the Amazon EBS instance attached to instance running this Jupyter notebook. However, Amazon Personalize will need an S3 bucket to act as the source of your data, as well as IAM roles for accessing that bucket. Let's set all of that up.

Use the metadata stored on the instance underlying this Amazon SageMaker notebook, to determine the region it is operating in. If you are using a Jupyter notebook outside of Amazon SageMaker, simply define the region as a string below. The Amazon S3 bucket needs to be in the same region as the Amazon Personalize resources we have been creating so far.

In [288]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


Amazon S3 bucket names are globally unique. To create a unique bucket name, the code below will append the string `personalizepoc` to your AWS account number. Then it creates a bucket with this name in the region discovered in the previous cell.

In [289]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
suffix = str(np.random.uniform())[4:9]
bucket_name = "personalize-hrnn-metadata-example" + suffix
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

personalize-hrnn-metadata-example05646


### Upload data to S3

Now that your Amazon S3 bucket has been created, upload the CSV file of our user-item-interaction data. 

In [294]:
interactions_filename = data_dir + '/a_interactions.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

In [295]:
user_metadata_file = data_dir + '/a_users.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(user_metadata_file).upload_file(user_metadata_file)

## Create dataset groups and the interactions dataset <a class="anchor" id="group_dataset"></a>
[Back to top](#top)

The highest level of isolation and abstraction with Amazon Personalize is a *dataset group*. Information stored within one of these dataset groups has no impact on any other dataset group or models created from one - they are completely isolated. This allows you to run many experiments and is part of how we keep your models private and fully trained only on your data. 

Before importing the data prepared earlier, there needs to be a dataset group and a dataset added to it that handles the interactions.

Dataset groups can house the following types of information:

* User-item-interactions
* Event streams (real-time interactions)
* User metadata
* Item metadata

Before we create the dataset group and the dataset for our interaction data, let's validate that your environment can communicate successfully with Amazon Personalize.

In [293]:
personalize = boto3.client(service_name='personalize')
personalize_runtime = boto3.client(service_name='personalize-runtime')
personalize_events = boto3.client(service_name='personalize-events')

### Create a Dataset Group

The following cell will create a new dataset group with the name *airlines-dataset-group* + a suffix

In [64]:
dataset_group_name = "airlines-dataset-group-" + suffix

create_dataset_group_response = personalize.create_dataset_group(
    name = dataset_group_name
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:144386903708:dataset-group/airlines-dataset-group-55035",
  "ResponseMetadata": {
    "RequestId": "a8bb75fb-f15b-45da-997e-08eb14d7733a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:14:12 GMT",
      "x-amzn-requestid": "a8bb75fb-f15b-45da-997e-08eb14d7733a",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Before we can use the dataset group, it must be active. This can take a minute or two. Execute the cell below and wait for it to show the ACTIVE status. It checks the status of the dataset group every second, up to a maximum of 3 hours.

In [65]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


Now that you have a dataset group, you can create a dataset for the interaction data.

# Create datasets

### Interactions Dataset

First, define a schema to tell Amazon Personalize what type of dataset you are uploading. There are several reserved and mandatory keywords required in the schema, based on the type of dataset. More detailed information can be found in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Here, you will create a schema for interactions data, which needs the `USER_ID`, `ITEM_ID`, `TIMESTAMP`, `CABIN_TYPE`, `EVENT_TYPE`, `EVENT_VALUE`, and `TIMESTAMP` fields. These must be defined in the same order in the schema as they appear in the dataset.

In [55]:
schema_name="airlines-interaction-schema-"+suffix

In [56]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name":"CABIN_TYPE",
            "type": "string",
            "categorical": True
        },
        {
          "name": "EVENT_TYPE",
          "type": "string"
        },
        {
          "name": "EVENT_VALUE",
          "type": "float"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = schema_name,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/airlines-interaction-schema-55035",
  "ResponseMetadata": {
    "RequestId": "4e045a61-d479-485c-93ff-6072076ccaa9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:10:34 GMT",
      "x-amzn-requestid": "4e045a61-d479-485c-93ff-6072076ccaa9",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [66]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn,
    name = "airlines-dataset-interactions-" + suffix
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:144386903708:dataset/airlines-dataset-group-55035/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "968e6cac-310a-4889-8243-e86ef90696ed",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:15:14 GMT",
      "x-amzn-requestid": "968e6cac-310a-4889-8243-e86ef90696ed",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Users Dataset

Here, you will create a schema for the users data, which needs the `USER_ID`, and `NATIONALITY` fields. These must be defined in the same order in the schema as they appear in the dataset.


In [62]:
metadata_schema_name="airlines-users-schema-"+suffix

In [63]:
metadata_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "NATIONALITY",
            "type": "string",
            "categorical": True
        }
    ],
    "version": "1.0"
}

create_metadata_schema_response = personalize.create_schema(
    name = metadata_schema_name,
    schema = json.dumps(metadata_schema)
)

metadata_schema_arn = create_metadata_schema_response['schemaArn']
print(json.dumps(create_metadata_schema_response, indent=2))


{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/airlines-users-schema-55035",
  "ResponseMetadata": {
    "RequestId": "17844e2f-860d-484a-bb39-ab4a10e7b9fd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:13:50 GMT",
      "x-amzn-requestid": "17844e2f-860d-484a-bb39-ab4a10e7b9fd",
      "content-length": "93",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
dataset_type = "USERS"
create_metadata_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = metadata_schema_arn,
    name = "airlines-metadata-dataset-users-" + suffix
)

metadata_dataset_arn = create_metadata_dataset_response['datasetArn']
print(json.dumps(create_metadata_dataset_response, indent=2))

## Configure an S3 bucket and an IAM  role <a class="anchor" id="bucket_role"></a>

So far, we have downloaded, manipulated, and saved the data onto the Amazon EBS instance attached to instance running this Jupyter notebook. However, Amazon Personalize will need an S3 bucket to act as the source of your data, as well as IAM roles for accessing that bucket. Let's set all of that up.

Use the metadata stored on the instance underlying this Amazon SageMaker notebook, to determine the region it is operating in. If you are using a Jupyter notebook outside of Amazon SageMaker, simply define the region as a string below. The Amazon S3 bucket needs to be in the same region as the Amazon Personalize resources we have been creating so far.

### Set the S3 bucket policy
Amazon Personalize needs to be able to read the contents of your S3 bucket. So add a bucket policy which allows that.

In [69]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy));

### Create an IAM role

Amazon Personalize needs the ability to assume roles in AWS in order to have the permissions to execute certain tasks. Let's create an IAM role and attach the required policies to it. The code below attaches very permissive policies; please use more restrictive policies for any production application.

In [70]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise
        
# sometimes need to wait a bit for the role to be created
time.sleep(45)
print(role_arn)

arn:aws:iam::144386903708:role/PersonalizeS3Role-55035


# Create your Dataset import jobs

## Import the interactions data <a class="anchor" id="import"></a>

Earlier you created the dataset group and dataset to house your information, so now you will execute an import job that will load the data from the S3 bucket into the Amazon Personalize dataset. 

In [109]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "airlines-dataset-import-job-"+suffix,
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/airlines-dataset-import-job-14078",
  "ResponseMetadata": {
    "RequestId": "d118cf11-b568-4767-99d5-30a15871981a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:46:38 GMT",
      "x-amzn-requestid": "d118cf11-b568-4767-99d5-30a15871981a",
      "content-length": "121",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Import the Users data <a class="anchor" id="import"></a>

Earlier you created the dataset group and dataset to house your information, so now you will execute an import job that will load the data from the S3 bucket into the Amazon Personalize dataset. 

In [110]:
create_metadata_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "airlines-users-metadata-dataset-import-job-"+suffix,
    datasetArn = metadata_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, user_metadata_file)
    },
    roleArn = role_arn
)

metadata_dataset_import_job_arn = create_metadata_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_metadata_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/airlines-users-metadata-dataset-import-job-14078",
  "ResponseMetadata": {
    "RequestId": "679d9401-568d-45e0-ba8c-df8f1574228d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:46:40 GMT",
      "x-amzn-requestid": "679d9401-568d-45e0-ba8c-df8f1574228d",
      "content-length": "136",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for the Dataset Import Jobs to have ACTIVE Status

Before we can use the dataset, the import job must be active. Execute the cell below and wait for it to show the ACTIVE status. It checks the status of the import job every second, up to a maximum of 3 hours.

Importing the data can take some time, depending on the size of the dataset. In this demo, the data import job has been pre executed.

In [111]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [112]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = metadata_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


When the dataset import is active, you are ready to start building models with HRNN-Metadata recipes.

## Create solutions <a class="anchor" id="solutions"></a>
[Back to top](#top)

In this notebook, we will create a solution with the following recipe:

1. HRNN-Metadata


In Amazon Personalize, a specific variation of an algorithm is called a recipe. Different recipes are suitable for different situations. A trained model is called a solution, and each solution can have many versions that relate to a given volume of data when the model was trained.

To start, we will list all the recipes that are supported. This will allow you to select one and use that to build your model.

In [113]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims
arn:aws:personalize:::recipe/aws-user-personalization


The output is just a JSON representation of all of the algorithms mentioned in the introduction.

Next we will select specific recipes and build models with them.

### HRNN-Metadata

HRNN (hierarchical recurrent neural network) is one of the more advanced recommendation models that you can use and it allows for real-time updates of recommendations based on user behavior. It also tends to outperform other approaches, like collaborative filtering. This recipe takes the longest to train, so let's start with this recipe first.

For our use case, using the Airlines reviews data, we can use the HRNN-Metadata to recommend airlines to a user based on the user's previous artist tagging behavior. Remember, we used the tagging data to represent positive interactions between a user and an artist.

First, select the recipe by finding the ARN in the list of recipes above.

In [114]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn-metadata"

#### Create the solution

First you create a solution using the recipe. Although you provide the dataset ARN in this step, the model is not yet trained. See this as an identifier instead of a trained model.

Note that we have HPO activated here. This is a good idea when 


In [116]:
create_solution_response = personalize.create_solution(
    name = "airlines-hrnn-metadata-solution-HPO-"+suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
    performHPO=True
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:144386903708:solution/airlines-hrnn-metadata-solution-HPO-14078",
  "ResponseMetadata": {
    "RequestId": "f25d8556-6745-453a-b0c4-5380b2da097b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 22:01:31 GMT",
      "x-amzn-requestid": "f25d8556-6745-453a-b0c4-5380b2da097b",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create the solution version

Once you have a solution, you need to create a version in order to complete the model training. The training can take a while to complete, upwards of 25 minutes, and an average of 40 minutes for this recipe with our dataset. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create many models and deploy them quickly. So we will set up the while loop for all of the solutions further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [117]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/airlines-hrnn-metadata-solution-HPO-14078/54a6c563",
  "ResponseMetadata": {
    "RequestId": "148a0fbd-5465-4619-ac90-449c0ef23b73",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 22:01:32 GMT",
      "x-amzn-requestid": "148a0fbd-5465-4619-ac90-449c0ef23b73",
      "content-length": "127",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [118]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

## Evaluate solution versions <a class="anchor" id="eval"></a>
[Back to top](#top)

It should not take more than an hour to train all the solutions from this notebook. While training is in progress, we recommend taking the time to read up on the various algorithms (recipes) and their behavior in detail. This is also a good time to consider alternatives to how the data was fed into the system and what kind of results you expect to see.

When the solutions finish creating, the next step is to obtain the evaluation metrics. Personalize calculates these metrics based on a subset of the training data. The image below illustrates how Personalize splits the data. Given 10 users, with 10 interactions each (a circle represents an interaction), the interactions are ordered from oldest to newest based on the timestamp. Personalize uses all of the interaction data from 90% of the users (blue circles) to train the solution version, and the remaining 10% for evaluation. For each of the users in the remaining 10%, 90% of their interaction data (green circles) is used as input for the call to the trained model. The remaining 10% of their data (orange circle) is compared to the output produced by the model and used to calculate the evaluation metrics.

![personalize metrics](static/imgs/personalize_metrics.png)

We recommend reading [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) to understand the metrics, but we have also copied parts of the documentation below for convenience.

You need to understand the following terms regarding evaluation in Personalize:

* *Relevant recommendation* refers to a recommendation that matches a value in the testing data for the particular user.
* *Rank* refers to the position of a recommended item in the list of recommendations. Position 1 (the top of the list) is presumed to be the most relevant to the user.
* *Query* refers to the internal equivalent of a GetRecommendations call.

The metrics produced by Personalize are:
- **coverage**: The proportion of unique recommended items from all queries out of the total number of unique items in the training data (includes both the Items and Interactions datasets).
- **mean_reciprocal_rank_at_25**: The [mean of the reciprocal ranks](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) of the first relevant recommendation out of the top 25 recommendations over all queries. This metric is appropriate if you're interested in the single highest ranked recommendation.
- **normalized_discounted_cumulative_gain_at_K**: Discounted gain assumes that recommendations lower on a list of recommendations are less relevant than higher recommendations. Therefore, each recommendation is discounted (given a lower weight) by a factor dependent on its position. To produce the [cumulative discounted gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) at K, each relevant discounted recommendation in the top K recommendations is summed together. The normalized discounted cumulative gain (NDCG) is the DCG divided by the ideal DCG such that NDCG is between 0 - 1. (The ideal DCG is where the top K recommendations are sorted by relevance.) Amazon Personalize uses a weighting factor of 1/log(1 + position), where the top of the list is position 1. This metric rewards relevant items that appear near the top of the list, because the top of a list usually draws more attention.
- **precision_at_K**: The number of relevant recommendations out of the top K recommendations divided by K. This metric rewards precise recommendation of the relevant items.

Let's take a look at the evaluation metrics for each of the solutions produced in this notebook. *Please note, your results might differ from the results described in the text of this notebook, due to the quality of the LastFM dataset.* 

### HRNN-Metadata metrics

First, retrieve the evaluation metrics for the HRNN solution version.

In [120]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))


{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/airlines-hrnn-metadata-solution-HPO-14078/54a6c563",
  "metrics": {
    "coverage": 0.4046,
    "mean_reciprocal_rank_at_25": 0.2035,
    "normalized_discounted_cumulative_gain_at_10": 0.2909,
    "normalized_discounted_cumulative_gain_at_25": 0.3174,
    "normalized_discounted_cumulative_gain_at_5": 0.2418,
    "precision_at_10": 0.0444,
    "precision_at_25": 0.022,
    "precision_at_5": 0.0605
  },
  "ResponseMetadata": {
    "RequestId": "156a6e70-152b-4940-8b17-048252419fd0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 23:02:24 GMT",
      "x-amzn-requestid": "156a6e70-152b-4940-8b17-048252419fd0",
      "content-length": "424",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


# Create a campaign from the solution

## Create campaigns <a class="anchor" id="create"></a>

A campaign is a hosted solution version; an endpoint which you can query for recommendations. Pricing is set by estimating throughput capacity (requests from users for personalization per second). When deploying a campaign, you set a minimum throughput per second (TPS) value. This service, like many within AWS, will automatically scale based on demand, but if latency is critical, you may want to provision ahead for larger demand. For this POC and demo, all minimum throughput thresholds are set to 1. For more information, see the [pricing page](https://aws.amazon.com/personalize/pricing/).

Let's start deploying the campaigns.

### HRNN-Metadata

Deploy a campaign for your HRNN solution version. It can take around 10 minutes to deploy a campaign. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create multiple campaigns. So we will set up the while loop for all of the campaigns further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [122]:
create_campaign_response = personalize.create_campaign(
    name = "airlines-metadata-campaign-"+suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 2,    
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:144386903708:campaign/airlines-metadata-campaign-14078",
  "ResponseMetadata": {
    "RequestId": "4c882630-86aa-4c5d-accd-75225f9804a4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 23:25:37 GMT",
      "x-amzn-requestid": "4c882630-86aa-4c5d-accd-75225f9804a4",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [123]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


### HRNN

HRNN is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users.

In [262]:
users_df = pd.read_csv('a_users.csv')
# Render some sample data
users_df.sample(5)

,USER_ID,NATIONALITY
3918,JHartley,United Kingdom
27477,DDriscoll,United Kingdom
19563,BrianElliott,United Kingdom
22989,AHornbuckle,Australia
35724,CMoon,United Kingdom


Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [165]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    
#   Context Recommendations
    context_options = ['None','Economy', 'Business Class','Premium Economy', 'First Class']
    
    for context in context_options:
        # Get the recommendations
        if context=='none':
            get_recommendations_response = personalize_runtime.get_recommendations(
                campaignArn = campaign_arn,
                userId = str(user_id),
            )
        else:
            get_recommendations_response = personalize_runtime.get_recommendations(
                campaignArn = campaign_arn,
                userId = str(user_id),
                context = {
                  'CABIN_TYPE': context
                }
            )
        # Build a new dataframe of recommendations
        item_list = get_recommendations_response['itemList']
        recommendation_list = []
        for item in item_list:
            recommendation_list.append(item['itemId'])
    #     print(recommendation_list)
        new_rec_DF = pd.DataFrame(recommendation_list, columns = [context])
        # Add this dataframe to the old one
        recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [263]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

      USER_ID     NATIONALITY
37013    RDow  United Kingdom


,None,Economy,Business Class,Premium Economy,First Class
0,british-airways,thomson-airways,british-airways,thomson-airways,united-airlines
1,united-airlines,thomas-cook-airlines,virgin-atlantic-airways,virgin-atlantic-airways,british-airways
2,thomson-airways,united-airlines,turkish-airlines,air-new-zealand,american-airlines
3,virgin-atlantic-airways,easyjet,united-airlines,british-airways,china-southern-airlines
4,air-new-zealand,monarch-airlines,china-southern-airlines,united-airlines,delta-air-lines
5,turkish-airlines,virgin-atlantic-airways,qatar-airways,thomas-cook-airlines,lufthansa
6,china-southern-airlines,british-airways,emirates,turkish-airlines,alaska-airlines
7,thomas-cook-airlines,jet2-com,air-france,monarch-airlines,virgin-atlantic-airways
8,lufthansa,lufthansa,american-airlines,china-southern-airlines,us-airways
9,american-airlines,american-airlines,lufthansa,eva-air,thomson-airways


In [264]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

      USER_ID NATIONALITY
26198   CJeff   Singapore


,None,Economy,Business Class,Premium Economy,First Class
0,philippine-airlines,philippine-airlines,cathay-pacific-airways,cathay-pacific-airways,singapore-airlines
1,cathay-pacific-airways,singapore-airlines,philippine-airlines,air-france,thai-airways
2,singapore-airlines,tigerair,malaysia-airlines,eva-air,philippine-airlines
3,ana-all-nippon-airways,jetstar-asia,srilankan-airlines,air-new-zealand,delta-air-lines
4,thai-airways,cathay-pacific-airways,thai-airways,philippine-airlines,emirates
5,air-india,airasia,singapore-airlines,klm-royal-dutch-airlines,ana-all-nippon-airways
6,china-eastern-airlines,cebu-pacific,air-india,airasia-x,china-southern-airlines
7,dragonair,scoot,emirates,qantas-airways,alaska-airlines
8,srilankan-airlines,ana-all-nippon-airways,ana-all-nippon-airways,china-southern-airlines,american-airlines
9,air-france,dragonair,asiana-airlines,united-airlines,china-eastern-airlines


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

#### Real time events

Start by creating an event tracker that is attached to the campaign.

In [150]:
response = personalize.create_event_tracker(
    name='AirlinesEventsTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:144386903708:event-tracker/d2e7ccdc
820029aa-b00c-4eff-9e6f-60830bb68508


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [200]:
session_dict = {}

def send_user_rating(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
        "itemId": str(ITEM_ID),
        "eventValue": 10,
        "cabinType": "Economy"
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
        trackingId = TRACKING_ID,
        userId= str(USER_ID),
        sessionId = session_ID,
        eventList = [{
            'sentAt': int(time.time()),
            'eventType': 'RATING',
            'properties': event_json
            }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id):
    # Interact with the airline
    # Sending a rating of 10 in Economy class for the airline with that user
    send_user_rating(USER_ID=user_id, ITEM_ID=item_id)
    
    
    #   Context Recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
        context = {
          'CABIN_TYPE': 'Economy'
        }
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        recommendation_list.append(item['itemId'])
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [item_id+'|Economy'])
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df


At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

## Recommendations before using the Event Tracker

In [265]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)
user_id = users[0]
recommendations_df_users

        USER_ID NATIONALITY
6313  ACrociani       Italy


,None,Economy,Business Class,Premium Economy,First Class
0,british-airways,alitalia,iberia,british-airways,american-airlines
1,alitalia,brussels-airlines,british-airways,virgin-atlantic-airways,british-airways
2,iberia,ryanair,qatar-airways,united-airlines,delta-air-lines
3,brussels-airlines,easyjet,alitalia,brussels-airlines,united-airlines
4,qatar-airways,iberia,brussels-airlines,turkish-airlines,lufthansa
5,lufthansa,aer-lingus,emirates,alitalia,emirates
6,american-airlines,aegean-airlines,lufthansa,air-france,qatar-airways
7,united-airlines,lufthansa,turkish-airlines,lufthansa,swiss-international-air-lines
8,icelandair,qatar-airways,swiss-international-air-lines,icelandair,us-airways
9,delta-air-lines,tap-portugal,oman-air,delta-air-lines,iberia


In [266]:
user_id

'ACrociani'

Ok, so now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [267]:
# Next generate 3 random Airlines
airlines = a_interactions_df.sample(3)['ITEM_ID'].tolist()
airlines

['thai-airways', 'austrian-airlines', 'united-airlines']

In [268]:
user_recommendations_df = pd.DataFrame()
# Note this will take about 15 seconds to complete due to the sleeps
for airline in airlines:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, airline)
    time.sleep(5)
print(user_id)
user_recommendations_df

ACrociani


,thai-airways|Economy,austrian-airlines|Economy,united-airlines|Economy
0,alitalia,ryanair,brussels-airlines
1,brussels-airlines,brussels-airlines,lufthansa
2,ryanair,easyjet,turkish-airlines
3,easyjet,tap-portugal,austrian-airlines
4,iberia,aegean-airlines,tap-portugal
5,aer-lingus,turkish-airlines,alitalia
6,aegean-airlines,lufthansa,germanwings
7,lufthansa,alitalia,aegean-airlines
8,qatar-airways,iberia,swiss-international-air-lines
9,tap-portugal,air-india,air-berlin


In the cell above, the first column after the index is the user's default recommendations from HRNN-Metadata, and each column after that has a header of the airlines that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much; this is due to the relatively limited nature of this dataset. If you wanted to better understand this, try simulating rating random airlines with random ratings, and you should see a more pronounced impact.